# Feature Importance (LGBM)

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/lg-aimers/sample_submission.csv
/kaggle/input/lg-aimers/train.csv
/kaggle/input/lg-aimers/test.csv


In [2]:
data_path = '/kaggle/input/lg-aimers/'

train = pd.read_csv(data_path + 'train.csv', index_col= 'ID')
test = pd.read_csv(data_path + 'test.csv', index_col= 'ID')
submission = pd.read_csv(data_path + 'sample_submission.csv', index_col= 'ID')

## 1. 데이터 통합

In [3]:
all_data = pd.concat([train, test], ignore_index= True)
all_data = all_data.drop('임신 성공 여부', axis= 1)

## 2. 피처 엔지니어링

In [4]:
drop_features = []

In [5]:
# "시술 시기 코드", "특정 시술 유형", "배아 생성 주요 이유" 제거
# "정자 출처", "정자 기증자 나이" 제거 -> 제외
# "주요 시술 유형" 생성

cat_features = [ 
    "시술 당시 나이", # FI top4
    "시술 유형",
    "주요 시술 유형",
    "배란 유도 유형",
    "난자 출처", # FI top
        
    
    "난자 기증자 나이", #FI top (알수 없음)

    "정자 출처",
    "정자 기증자 나이",
    "시술 시기 코드", 
    "배아 생성 주요 이유"
]

print(len(cat_features))

drop_features = drop_features + ["특정 시술 유형"]

10


In [6]:
# "IVF 시술 횟수", "IVF 임신 횟수", "IVF 출산 횟수" 제거
# "DI 시술 횟수", "DI 임신 횟수", "DI 출산 횟수" 제거
# "IVF 임신 성공률", "IVF 출산 성공률", "IVF 실패 횟수", "IVF 유산 횟수", "DI 임신 성공률", "DI 출산 성공률", "DI 실패 횟수", "DI 유산 횟수" 생성 

ord_features = [
    "IVF 임신 성공률", 
    "IVF 출산 성공률", 
    "IVF 실패 횟수", 
    "IVF 유산 횟수", 
    "DI 임신 성공률", 
    "DI 출산 성공률", 
    "DI 실패 횟수", 
    "DI 유산 횟수",
    
    "클리닉 내 총 시술 횟수", #FI top
    "총 시술 횟수",
    "총 임신 횟수",
    "총 출산 횟수"
]

print(len(ord_features))

# drop_features = drop_features + ["총 시술 횟수", "총 임신 횟수", "총 출산 횟수"]

12


In [7]:
# "난자 해동 경과일", "난자 혼합 경과일", "배아 해동 경과일" 제거 ->제외 
# "저장된 신선 난자 수", "기증자 정자와 혼합된 난자 수" 제거 -> 제외

num_features = [
    "이식된 배아 수", # FI top1
    "해동 난자 수", #FI top
    "수집된 신선 난자 수", #FI top7
    "파트너 정자와 혼합된 난자 수", #FI top
    "배아 이식 경과일", # FI top2,
    
    
    "저장된 배아 수", # FI top3
    "미세주입 후 저장된 배아 수", #FI top
    "총 생성 배아 수", # FI top5
    "미세주입된 난자 수", #FI top8
    "혼합된 난자 수", #FI top
    "미세주입에서 생성된 배아 수", #FI top
    "미세주입 배아 이식 수", #FI top
    "해동된 배아 수", #FI top
    "난자 채취 경과일", #FI top
    "임신 시도 또는 마지막 임신 경과 연수", #FI top

    "난자 해동 경과일",
    "난자 혼합 경과일",
    "배아 해동 경과일",
    "저장된 신선 난자 수",
    "기증자 정자와 혼합된 난자 수"
]

print(len(num_features))

# drop_features = drop_features + ["난자 해동 경과일", "난자 혼합 경과일", "배아 해동 경과일", "저장된 신선 난자 수", "기증자 정자와 혼합된 난자 수"]

20


In [8]:
# "남성 주 불임 원인", "남성 부 불임 원인", "불임 원인 - 남성 요인", "불임 원인 - 정자 농도", "불임 원인 - 정자 면역학적 요인", "불임 원인 - 정자 운동성", "불임 원인 - 정자 형태",
# "여성 주 불임 원인", "여성 부 불임 원인", "불임 원인 - 난관 질환", "불임 원인 - 배란 장애", "불임 원인 - 여성 요인", "불임 원인 - 자궁경부 문제", "불임 원인 - 자궁내막증" 제거
# "부부 주 불임 원인", "부부 부 불임 원인" 제거
# "불명확 불임 원인" 제거 -> 제외
# "남성 불임 원인", "여성 불임 원인", "부부 불임 원인" 생성
# '착상 전 유전 검사 사용 여부', "기증 배아 사용 여부" 제거 -> 제외
# "대리모 여부", "PGD 시술 여부", "PGS 시술 여부" 제거 

bin_features = [
    '배란 자극 여부',
    '단일 배아 이식 여부', #FI top
    '착상 전 유전 진단 사용 여부',
    '남성 불임 원인',
    '여성 불임 원인',
    '부부 불임 원인',
    "동결 배아 사용 여부",
    "신선 배아 사용 여부",

    "불명확 불임 원인",
    "기증 배아 사용 여부",
    '착상 전 유전 검사 사용 여부',
    "대리모 여부"
]

print(len(bin_features))

drop_features = drop_features + ["PGD 시술 여부", "PGS 시술 여부"]

12


In [9]:
# 시술 유형(cat feature) 피처 엔지니어링 

all_data["주요 시술 유형"] = all_data["특정 시술 유형"].apply(lambda x: 
    "ICSI" if "ICSI" in str(x) else 
    "IVF" if "IVF" in str(x) else 
    "IUI" if "IUI" in str(x) else 
    "Other"
)

drop_features = drop_features + ['특정 시술 유형']

all_data['주요 시술 유형'].value_counts(normalize=True, dropna=False) * 100

주요 시술 유형
ICSI     50.208130
IVF      36.909168
Other    10.511578
IUI       2.371124
Name: proportion, dtype: float64

In [10]:
# 불임 원인(bin features) 피처 엔지니어링 

infertility_features = ["남성 주 불임 원인", "남성 부 불임 원인", "여성 주 불임 원인", "여성 부 불임 원인",
                        "불임 원인 - 남성 요인", "불임 원인 - 정자 농도", "불임 원인 - 정자 면역학적 요인", "불임 원인 - 정자 운동성", "불임 원인 - 정자 형태",
                        "불임 원인 - 난관 질환", "불임 원인 - 배란 장애", "불임 원인 - 여성 요인", "불임 원인 - 자궁경부 문제", "불임 원인 - 자궁내막증",
                        "부부 주 불임 원인", "부부 부 불임 원인"]

all_data["남성 불임 원인"] = (
    all_data["남성 주 불임 원인"] + 
    all_data["남성 부 불임 원인"] + 
    all_data["불임 원인 - 남성 요인"] + 
    all_data["불임 원인 - 정자 농도"] + 
    all_data["불임 원인 - 정자 면역학적 요인"] + 
    all_data["불임 원인 - 정자 운동성"] + 
    all_data["불임 원인 - 정자 형태"]
).apply(lambda x: 1 if x > 0 else 0)

all_data["여성 불임 원인"] = (
    all_data["여성 주 불임 원인"] + 
    all_data["여성 부 불임 원인"] + 
    all_data["불임 원인 - 난관 질환"] + 
    all_data["불임 원인 - 배란 장애"] + 
    all_data["불임 원인 - 여성 요인"] + 
    all_data["불임 원인 - 자궁경부 문제"] + 
    all_data["불임 원인 - 자궁내막증"]
).apply(lambda x: 1 if x > 0 else 0)

all_data["부부 불임 원인"] = (
    all_data["부부 주 불임 원인"] + 
    all_data["부부 부 불임 원인"]
).apply(lambda x: 1 if x > 0 else 0)

drop_features = drop_features + infertility_features

print(all_data['남성 불임 원인'].value_counts(normalize=True, dropna=False) * 100)
print(all_data['여성 불임 원인'].value_counts(normalize=True, dropna=False) * 100)
print(all_data['부부 불임 원인'].value_counts(normalize=True, dropna=False) * 100)

남성 불임 원인
0    60.57855
1    39.42145
Name: proportion, dtype: float64
여성 불임 원인
0    66.416583
1    33.583417
Name: proportion, dtype: float64
부부 불임 원인
0    95.81546
1     4.18454
Name: proportion, dtype: float64


In [11]:
# 과거 이력(ord feature) 피처 엔지니어링

past_features = ["IVF 시술 횟수", "DI 시술 횟수", "IVF 임신 횟수", "DI 임신 횟수", "IVF 출산 횟수", "DI 출산 횟수"]

for col in ["총 시술 횟수", "총 임신 횟수", "총 출산 횟수", "클리닉 내 총 시술 횟수"]:
    all_data[col] = all_data[col].str.replace("회 이상", "").str.replace("회", "").astype(float)
        
for col in past_features:
    all_data[col] = all_data[col].str.replace("회 이상", "").str.replace("회", "").astype(float)

all_data["IVF 임신 성공률"] = all_data["IVF 임신 횟수"] / (all_data["IVF 시술 횟수"] + 1)
all_data["IVF 출산 성공률"] = all_data["IVF 출산 횟수"] / (all_data["IVF 임신 횟수"] + 1)
all_data["IVF 실패 횟수"] = all_data["IVF 시술 횟수"] - all_data["IVF 임신 횟수"]
all_data["IVF 유산 횟수"] = all_data["IVF 임신 횟수"] - all_data["IVF 출산 횟수"]

all_data["DI 임신 성공률"] = all_data["DI 임신 횟수"] / (all_data["DI 시술 횟수"] + 1)
all_data["DI 출산 성공률"] = all_data["DI 출산 횟수"] / (all_data["DI 임신 횟수"] + 1)
all_data["DI 실패 횟수"] = all_data["DI 시술 횟수"] - all_data["DI 임신 횟수"]
all_data["DI 유산 횟수"] = all_data["DI 임신 횟수"] - all_data["DI 출산 횟수"]

drop_features = drop_features + past_features

# print(all_data['IVF 임신 성공률'].value_counts(normalize=True, dropna=False) * 100)
# print(all_data["IVF 출산 성공률"].value_counts(normalize=True, dropna=False) * 100)
# print(all_data["IVF 실패 횟수"].value_counts(normalize=True, dropna=False) * 100)
# print(all_data["IVF 유산 횟수"].value_counts(normalize=True, dropna=False) * 100)
# print(all_data["DI 임신 성공률"].value_counts(normalize=True, dropna=False) * 100)
# print(all_data["DI 출산 성공률"].value_counts(normalize=True, dropna=False) * 100)
# print(all_data["DI 실패 횟수"].value_counts(normalize=True, dropna=False) * 100)
# print(all_data["DI 유산 횟수"].value_counts(normalize=True, dropna=False) * 100)

# print(all_data["총 시술 횟수"].value_counts(normalize=True, dropna=False) * 100)
# print(all_data["총 임신 횟수"].value_counts(normalize=True, dropna=False) * 100)
# print(all_data["총 출산 횟수"].value_counts(normalize=True, dropna=False) * 100)
# print(all_data["IVF 임신 횟수"].value_counts(normalize=True, dropna=False) * 100)

In [12]:
extracted_data = all_data.drop(columns=drop_features, errors='ignore')

In [13]:
extracted_features = extracted_data.columns
extracted_features

Index(['시술 시기 코드', '시술 당시 나이', '임신 시도 또는 마지막 임신 경과 연수', '시술 유형', '배란 자극 여부',
       '배란 유도 유형', '단일 배아 이식 여부', '착상 전 유전 검사 사용 여부', '착상 전 유전 진단 사용 여부',
       '불명확 불임 원인', '배아 생성 주요 이유', '총 시술 횟수', '클리닉 내 총 시술 횟수', '총 임신 횟수',
       '총 출산 횟수', '총 생성 배아 수', '미세주입된 난자 수', '미세주입에서 생성된 배아 수', '이식된 배아 수',
       '미세주입 배아 이식 수', '저장된 배아 수', '미세주입 후 저장된 배아 수', '해동된 배아 수', '해동 난자 수',
       '수집된 신선 난자 수', '저장된 신선 난자 수', '혼합된 난자 수', '파트너 정자와 혼합된 난자 수',
       '기증자 정자와 혼합된 난자 수', '난자 출처', '정자 출처', '난자 기증자 나이', '정자 기증자 나이',
       '동결 배아 사용 여부', '신선 배아 사용 여부', '기증 배아 사용 여부', '대리모 여부', '난자 채취 경과일',
       '난자 해동 경과일', '난자 혼합 경과일', '배아 이식 경과일', '배아 해동 경과일', '주요 시술 유형',
       '남성 불임 원인', '여성 불임 원인', '부부 불임 원인', 'IVF 임신 성공률', 'IVF 출산 성공률',
       'IVF 실패 횟수', 'IVF 유산 횟수', 'DI 임신 성공률', 'DI 출산 성공률', 'DI 실패 횟수',
       'DI 유산 횟수'],
      dtype='object')

In [14]:
len(extracted_features)

54

## 2. 피처 엔지니어링: 원 핫 인코딩

In [15]:
from sklearn.preprocessing import OneHotEncoder

onehot_encoder = OneHotEncoder()

encoded_cat_matrix = onehot_encoder.fit_transform(extracted_data[cat_features])

encoded_cat_matrix

<346418x57 sparse matrix of type '<class 'numpy.float64'>'
	with 3464180 stored elements in Compressed Sparse Row format>

In [16]:
remaining_features = list(set(extracted_features) - set(cat_features))

In [17]:
print(extracted_data[remaining_features].dtypes)

저장된 신선 난자 수              float64
난자 해동 경과일                float64
IVF 임신 성공률               float64
미세주입 후 저장된 배아 수          float64
미세주입에서 생성된 배아 수          float64
불명확 불임 원인                  int64
IVF 실패 횟수                float64
해동된 배아 수                 float64
혼합된 난자 수                 float64
배아 해동 경과일                float64
착상 전 유전 검사 사용 여부         float64
저장된 배아 수                 float64
여성 불임 원인                   int64
DI 유산 횟수                 float64
DI 출산 성공률                float64
단일 배아 이식 여부              float64
총 생성 배아 수                float64
파트너 정자와 혼합된 난자 수         float64
수집된 신선 난자 수              float64
배아 이식 경과일                float64
이식된 배아 수                 float64
DI 임신 성공률                float64
미세주입 배아 이식 수             float64
배란 자극 여부                   int64
착상 전 유전 진단 사용 여부         float64
기증 배아 사용 여부              float64
총 시술 횟수                  float64
IVF 출산 성공률               float64
임신 시도 또는 마지막 임신 경과 연수    float64
미세주입된 난자 수               float64
부부 불임 원인  

In [18]:
from scipy import sparse

extracted_data_sprs = sparse.hstack([sparse.csr_matrix(extracted_data[remaining_features].fillna(0)),
                               encoded_cat_matrix],
                              format='csr')

In [19]:
extracted_encoded_features = list(remaining_features) + list(onehot_encoder.get_feature_names_out(cat_features))

## 3. 데이터 나누기

In [20]:
num_train = len(train) 

X = extracted_data_sprs[:num_train]
X_test = extracted_data_sprs[num_train:]

y = train['임신 성공 여부'].values

## 4. ROC-AUC 정의

In [21]:
from sklearn.metrics import roc_auc_score

# LGBM 커스텀 평가 함수
def lgb_roc_auc(y_pred, dataset):
    y_true = dataset.get_label()
    return "roc_auc", roc_auc_score(y_true, y_pred), True  # (지표 이름, 값, 높은 값이 더 좋은지 여부)

## 5. 모델 훈련 

In [22]:
import lightgbm as lgb
from sklearn.model_selection import train_test_split

# 8:2 비율로 훈련 데이터, 검증 데이터 분리 (베이지안 최적화 수행용)
X_train, X_valid, y_train, y_valid = train_test_split(X, y, 
                                                      test_size=0.2, 
                                                      random_state=0)

# 베이지안 최적화용 데이터셋
bayes_dtrain = lgb.Dataset(X_train, y_train)
bayes_dvalid = lgb.Dataset(X_valid, y_valid)

In [23]:
# 베이지안 최적화를 위한 하이퍼파라미터 범위
# TODO: 점점 줄여나가자!!!!
param_bounds = {
    'num_leaves': (20, 40),  
    'lambda_l1': (0, 0.5),  
    'lambda_l2': (0.8, 2),  
    'feature_fraction': (0.8, 1),  
    'bagging_fraction': (0.5, 1),  
    'min_child_samples': (5, 25),  
    'min_child_weight': (25, 40)  
}

# 값이 고정된 하이퍼파라미터
fixed_params = {'objective': 'binary', # binary classification
                'learning_rate': 0.005, # 0.01~0.001
                'bagging_freq': 1, # 0 or 1
                'force_row_wise': True,
                'random_state': 1991}

In [24]:
from sklearn.metrics import roc_auc_score
from lightgbm import early_stopping

def eval_function(num_leaves, lambda_l1, lambda_l2, feature_fraction,
                  bagging_fraction, min_child_samples, min_child_weight):
    '''최적화하려는 평가지표 계산 함수'''
    
    # 베이지안 최적화를 수행할 하이퍼파라미터 
    params = {'num_leaves': int(round(num_leaves)),
              'lambda_l1': lambda_l1,
              'lambda_l2': lambda_l2,
              'feature_fraction': feature_fraction,
              'bagging_fraction': bagging_fraction,
              'min_child_samples': int(round(min_child_samples)),
              'min_child_weight': min_child_weight,
              'feature_pre_filter': False}
    # 고정된 하이퍼파라미터도 추가
    params.update(fixed_params)
    
    print('하이퍼파라미터:', params)    
    
    # LightGBM 모델 훈련
    lgb_model = lgb.train(params=params, 
                           train_set=bayes_dtrain,
                           num_boost_round=2500,
                           valid_sets=bayes_dvalid,
                           callbacks=[early_stopping(stopping_rounds=100)])
    # 검증 데이터로 예측 수행
    preds = lgb_model.predict(X_valid) 
    # roc-auc 계산
    roc_auc = roc_auc_score(y_valid, preds)
    print(f'roc-auc : {roc_auc}\n')
    
    return roc_auc

In [25]:
from bayes_opt import BayesianOptimization

# 베이지안 최적화 객체 생성
optimizer = BayesianOptimization(f=eval_function,      # 평가지표 계산 함수
                                 pbounds=param_bounds, # 하이퍼파라미터 범위
                                 random_state=0)

In [26]:
# 베이지안 최적화 수행
# TODO: init_points 10~15, n_iter 30~70
optimizer.maximize(init_points=10, n_iter=50)

|   iter    |  target   | baggin... | featur... | lambda_l1 | lambda_l2 | min_ch... | min_ch... | num_le... |
-------------------------------------------------------------------------------------------------------------
하이퍼파라미터: {'num_leaves': 29, 'lambda_l1': 0.30138168803582194, 'lambda_l2': 1.4538598195962762, 'feature_fraction': 0.943037873274484, 'bagging_fraction': 0.7744067519636624, 'min_child_samples': 13, 'min_child_weight': 34.68841169599984, 'feature_pre_filter': False, 'objective': 'binary', 'learning_rate': 0.005, 'bagging_freq': 1, 'force_row_wise': True, 'random_state': 1991}
[LightGBM] [Info] Number of positive: 52867, number of negative: 152213
[LightGBM] [Info] Total Bins 763
[LightGBM] [Info] Number of data points in the train set: 205080, number of used features: 100
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.257787 -> initscore=-1.057502
[LightGBM] [Info] Start training from score -1.057502
Training until validation scores don't improve for 100 rounds
Early

In [27]:
# 평가함수 점수가 최대일 때 하이퍼파라미터
max_params = optimizer.max['params']
max_params

{'bagging_fraction': 0.5676682710015916,
 'feature_fraction': 0.9534471026963212,
 'lambda_l1': 0.04811015026095178,
 'lambda_l2': 1.9483311603902702,
 'min_child_samples': 19.753033722267613,
 'min_child_weight': 35.098326662286894,
 'num_leaves': 24.44991218235853}

In [28]:
# 정수형 하이퍼파라미터 변환
max_params['num_leaves'] = int(round(max_params['num_leaves']))
max_params['min_child_samples'] = int(round(max_params['min_child_samples']))

# 값이 고정된 하이퍼파라미터 추가
max_params.update(fixed_params)

In [29]:
max_params

{'bagging_fraction': 0.5676682710015916,
 'feature_fraction': 0.9534471026963212,
 'lambda_l1': 0.04811015026095178,
 'lambda_l2': 1.9483311603902702,
 'min_child_samples': 20,
 'min_child_weight': 35.098326662286894,
 'num_leaves': 24,
 'objective': 'binary',
 'learning_rate': 0.005,
 'bagging_freq': 1,
 'force_row_wise': True,
 'random_state': 1991}

In [30]:
from sklearn.model_selection import StratifiedKFold

# 층화 K 폴드 교차 검증기 생성
folds = StratifiedKFold(n_splits=10, shuffle=True, random_state=1991)

In [31]:
def lgb_roc_auc(y_pred, dataset):
    y_true = dataset.get_label()
    return "roc_auc", roc_auc_score(y_true, y_pred), True  # (지표 이름, 값, 높은 값이 더 좋은지 여부)

In [32]:
import lightgbm as lgb

# OOF 방식으로 훈련된 모델로 검증 데이터 타깃값을 예측한 확률을 담을 1차원 배열
oof_val_preds_lgb = np.zeros(X.shape[0]) 
# OOF 방식으로 훈련된 모델로 테스트 데이터 타깃값을 예측한 확률을 담을 1차원 배열
oof_test_preds_lgb = np.zeros(X_test.shape[0]) 

# fold별 feature importance
fold_importance_df = pd.DataFrame()

# OOF 방식으로 모델 훈련, 검증, 예측
for idx, (train_idx, valid_idx) in enumerate(folds.split(X, y)):
    # 각 폴드를 구분하는 문구 출력
    print('#'*40, f'폴드 {idx+1} / 폴드 {folds.n_splits}', '#'*40)
    
    # 훈련용 데이터, 검증용 데이터 설정
    X_train, y_train = X[train_idx], y[train_idx] # 훈련용 데이터
    X_valid, y_valid = X[valid_idx], y[valid_idx] # 검증용 데이터

    # LightGBM 전용 데이터셋 생성
    dtrain = lgb.Dataset(X_train, y_train) # LightGBM 전용 훈련 데이터셋
    dvalid = lgb.Dataset(X_valid, y_valid) # LightGBM 전용 검증 데이터셋
                          
    # LightGBM 모델 훈련
    lgb_model = lgb.train(params=max_params,     # 최적 하이퍼파라미터
                          train_set=dtrain,          # 훈련 데이터셋
                          num_boost_round=2500,      # 부스팅 반복 횟수
                          valid_sets=dvalid,         # 성능 평가용 검증 데이터셋
                          feval = lgb_roc_auc,
                          callbacks=[lgb.early_stopping(stopping_rounds=100)])

    # 테스트 데이터를 활용해 OOF 예측
    oof_test_preds_lgb += lgb_model.predict(X_test)/folds.n_splits
    
    # 모델 성능 평가를 위한 검증 데이터 타깃값 예측 
    oof_val_preds_lgb[valid_idx] += lgb_model.predict(X_valid)

    # Feature Importance 저장
    fold_importance = pd.DataFrame({
        'Feature': extracted_encoded_features,
        'Importance': lgb_model.feature_importance(importance_type='gain'),
        'Fold': idx + 1
    })
    fold_importance_df = pd.concat([fold_importance_df, fold_importance], axis=0)
    
    # 검증 데이터 예측 확률에 대한 ROC-AUC
    roc_auc = roc_auc_score(y_valid, oof_val_preds_lgb[valid_idx])
    print(f'폴드 {idx+1} roc-auc : {roc_auc}\n')

######################################## 폴드 1 / 폴드 10 ########################################
[LightGBM] [Info] Number of positive: 59605, number of negative: 171110
[LightGBM] [Info] Total Bins 747
[LightGBM] [Info] Number of data points in the train set: 230715, number of used features: 91
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.258349 -> initscore=-1.054567
[LightGBM] [Info] Start training from score -1.054567
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[2252]	valid_0's binary_logloss: 0.487853	valid_0's roc_auc: 0.740125
폴드 1 roc-auc : 0.7401253641319613

######################################## 폴드 2 / 폴드 10 ########################################
[LightGBM] [Info] Number of positive: 59606, number of negative: 171110
[LightGBM] [Info] Total Bins 746
[LightGBM] [Info] Number of data points in the train set: 230716, number of used features: 91
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.258352 -> initscore=-1.054

In [33]:
print('OOF 검증 데이터 roc-auc :', roc_auc_score(y, oof_val_preds_lgb))

OOF 검증 데이터 roc-auc : 0.740382334703773


In [34]:
submission['posibility'] = oof_test_preds_lgb
submission.to_csv('alllll_submission.csv')

## 6. Feature Importance

In [35]:
import matplotlib.pyplot as plt
import matplotlib as mpl
import matplotlib.font_manager as fm
import seaborn as sns
import os

In [36]:
# !apt-get update -qq
# !apt-get install -qq -y fonts-nanum
# !fc-cache -fv
# !rm -rf ~/.cache/matplotlib

In [37]:
# 2️⃣ 한글 폰트 경로 설정
font_path = "/usr/share/fonts/truetype/nanum/NanumGothic.ttf"
font_prop = fm.FontProperties(fname=font_path)

# 3️⃣ Matplotlib에 한글 폰트 적용
mpl.rc('font', family=font_prop.get_name())  
mpl.rcParams['axes.unicode_minus'] = False  # 마이너스(-) 기호 깨짐 방지

# 4️⃣ 폰트가 제대로 적용됐는지 확인
plt.figure(figsize=(5, 2))
plt.text(0.5, 0.5, '한글 폰트 적용 확인!', fontsize=15, fontproperties=font_prop, ha='center')
plt.show()

FileNotFoundError: [Errno 2] No such file or directory: '/usr/share/fonts/truetype/nanum/NanumGothic.ttf'

In [ ]:
# 전체 Fold에서 평균 Feature Importance 계산
mean_importance = fold_importance_df.groupby("Feature")["Importance"].mean().reset_index()

# 중요도 순으로 정렬
mean_importance = mean_importance.sort_values(by="Importance", ascending=False)

def plot_feature_importance(feature_importance, top_n=50):
    """
    LightGBM Feature Importance 시각화 (상위 top_n개만)
    """
    # 전체 gain feature importance 총합
    print(feature_importance["Importance"].sum())

    # Gain이 0인 피처 제거
    feature_importance = feature_importance[feature_importance["Importance"] > 0]
    
    # 중요도가 높은 순으로 정렬 후 상위 N개 선택
    feature_importance = feature_importance.sort_values(by='Importance', ascending=False).head(top_n)

    # threshold = feature_importance["Importance"].sum() * 0.01  # 전체 Gain의 1%
    # feature_importance = feature_importance[feature_importance["Importance"] >= threshold]

    # 그래프 크기 설정
    plt.figure(figsize=(12, max(6, top_n // 5)))  # Feature 개수에 따라 동적 크기 조절
    
    # 가로 막대 그래프
    bars = plt.barh(feature_importance['Feature'], feature_importance['Importance'], color='skyblue')

    # 제목 & 라벨 설정 (한글 폰트 적용)
    plt.xlabel('Feature Importance (Mean Gain)', fontsize=12, fontproperties=font_prop)
    plt.ylabel('Features', fontsize=12, fontproperties=font_prop)
    plt.title(f'LightGBM Feature Importance (Top {top_n})', fontsize=14, fontproperties=font_prop)

    # 각 막대 앞부분에 Feature Importance 값 표시
    for bar, importance in zip(bars, feature_importance['Importance']):
        plt.text(bar.get_width() + feature_importance['Importance'].max() * 0.01,  # 오른쪽 여백 추가
                 bar.get_y() + bar.get_height()/2,  # 막대 중앙 정렬
                 f'{importance:,.0f}',  # 천 단위 콤마 추가
                 va='center',  # 수직 정렬
                 fontsize=10,
                 fontproperties=font_prop)

    # 중요도가 높은 순으로 보기 위해 Y축 반전
    plt.gca().invert_yaxis()
    
    # 레이아웃 조정
    plt.tight_layout()
    
    # 그래프 표시
    plt.show()


# Feature Importance 시각화 (상위 50개만)
plot_feature_importance(mean_importance, top_n=100)